# 大语言模型-Amazon Titan Text

同济子豪兄 2024-1-1

## 官方文档

https://docs.aws.amazon.com/bedrock/latest/userguide/titan-text-models.html

## 导入工具包

In [1]:
# Python 标准库
import json

# AWS 服务
import boto3
from botocore.config import Config

## 指定AWS Key

注册AWS IAM账号时生成的秘钥，需要换成你自己的

In [2]:
aws_access_key_id = 'XXXXXXX'
aws_secret_access_key = 'XXXXXXX'

## 指定AWS区域

默认使用`美国东部（弗吉尼亚北部）`，也就是`us-east-1`

In [3]:
aws_config = Config(
    region_name='us-east-1',
    proxies={},
)

## 创建Bedrock Client

In [4]:
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    config=aws_config,
)

/environment/miniconda3/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


## 选择大语言模型

In [55]:
modelId = 'amazon.titan-text-express-v1'

# modelId = 'amazon.titan-text-lite-v1'

## 一、单轮对话

### 提示词

In [56]:
prompt = '帮我写一句中国农历新年的问候语'

### 调用Amazon Bedrock基础模型API，生成回复

In [57]:
body = json.dumps({
    "inputText": prompt, 
    "textGenerationConfig":{  
        "maxTokenCount":512,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
    }
})

In [58]:
response = bedrock_runtime.invoke_model(
    body=body,
    modelId=modelId,
    accept="application/json", 
    contentType="application/json"
)

response_body = json.loads(response.get('body').read())
outputText = response_body.get('results')[0].get('outputText').strip()

### 查看大模型输出

In [59]:
print(outputText)

农历新年快乐！祝您生活幸福美满，万事如意！


## 二、多轮对话

### 背景提示词（世界观设定）

In [16]:
system_prompt = """让我们玩开放世界文本冒险游戏。你负责根据用户输入创建后续的游戏场景，以下是具体要求：
1. 这个游戏没有结局，所以请避免创造一个结局。
2. 在每一轮中，探索的过程要根据情景和上一轮的选择来想象。例如，“你进入了森林。这里又黑又诡异，什么地方都有可能跳出来奇怪的东西。你紧紧地握着手中的手枪，希望在这场冒险中找到一些食物”。
3. 探索所得的材料要与探索过程高度相关，比如在海里抓鱼，在沼泽地里获取沼气。
4. 你必须用一段话简短地描述每一轮的探索结果，内容包含探索的过程、探索所得的材料、探索的结果。
5. 请在用户说“开始”之后创建初始场景。
## 游戏背景 ##
数个世纪以前，这片大陆曾是一个繁荣的文明的家园。这个文明拥有强大的魔法和科技，创造出了许多神奇的事物，其中包括能够操控自然力量的神秘水晶。然而，在一场古老的灾难中，大陆被分隔成数个孤立的岛屿，并被一道神秘的能量屏障所封锁。这导致了文明的崩溃，人们的知识和技术都消失得无影无踪。
用户将扮演一位冒险家，听说在大陆的某个孤岛上，隐藏着一个传说中的古老遗迹，里面藏有能够打破能量屏障的秘密。用户的目标是探索这个孤岛，解开神秘谜团，寻找隐藏的线索，以揭示大陆背后的真相。
孤岛上充满了危险和奇异的生物，神秘的魔法和古老的机关。用户需要通过与岛上居民互动、解开谜题、战胜怪物等方式，逐渐揭开大陆灾难的真相，并寻找能够打破能量屏障的方法。游戏的故事情节充满扭曲的时空线索和出乎意料的转折，用户的每个决定都将影响游戏的发展和结局。
在探索的过程中，用户将收集各种资源、学习古老的魔法和技能，升级装备，以更好地应对岛上的挑战。同时，与岛上居民建立联系，听取他们的故事和建议，或许能够帮助用户更快地找到目标。

"""

### 之前的多轮对话提示词（人类用户和AI一问一答，并以AI回答作为结尾）

In [17]:
msg_list = [
    {
        "role": "User",
        "content": "开始",
    },
    {
        "role": "Bot",
        "content": """你发现自己身处一个被茂密丛林覆盖的小岛上。阳光透过树叶洒在大地上，空气中弥漫着新鲜的植物气息。岛上的树木参差不齐，地面上长满了各种奇异的花草。
你并不清楚自己是如何来到这个神秘的小岛的。唯一的线索是身边的一封古老信件，上面写着一位已故亲戚的名字，而信中提到了一个传说中的古老遗迹，藏匿在岛的深处。信件的最后，有一段文字：“只有真正的冒险者才能解开这个谜团，而你，将成为我们的希望”。""",
    },
    {
        "role": "User",
        "content": "进入丛林寻找可能的线索",
    },
    {
        "role": "Bot",
        "content": """你决定进入丛林寻找线索。在丛林中，你发现了一些奇怪的痕迹，似乎是一种未知的生物留下的。你跟踪着这些痕迹，来到了一处空地。空地中央有一块石碑，上面刻着一些奇怪的符号。你仔细观察了一下，发现这些符号似乎是一种古老的文字，但你并不认识。""",
    },
]

### 人类用户最新要说的提示词

In [18]:
prompt = '寻找岛上的居民询问石碑的来历'

### 调用Amazon Bedrock基础模型API，生成回复

In [19]:
msg_list.append({"role": "User", "content": prompt})
msg_list.append({"role": "Bot", "content": ""})
inputText = system_prompt + "\n\n".join(map(lambda x: f"{x['role']}: {x['content']}", msg_list))

In [20]:
body = json.dumps(
    {
        "inputText": inputText,
        "textGenerationConfig": {
            "maxTokenCount": 1024,
            "stopSequences": ["User:"],
            "temperature": 0.6,
        },
    }
)

In [21]:
response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept='application/json', contentType='application/json')
response_body = json.loads(response.get("body").read())
outputText = response_body.get("results")[0]["outputText"]

### 查看最新回复

In [23]:
print("Bot:", outputText)

Bot: 你找到了一位年轻的男子，他告诉你这块石碑是为了记录岛上的历史和传说而刻石的。他告诉你，岛上有一个古老的文明，拥有强大的魔法和科技，但在一场古老的灾难中被分隔成数个孤立的岛屿。他还告诉你，古老文明的传说中，有一种可以打破能量屏障的秘密，而这个秘密被寄托在古老遗迹中。




### 新的多轮对话提示词

In [ ]:
msg_list[-1]["content"] = outputText

In [25]:
msg_list

[{'role': 'User', 'content': '开始'},
 {'role': 'Bot',
  'content': '你发现自己身处一个被茂密丛林覆盖的小岛上。阳光透过树叶洒在大地上，空气中弥漫着新鲜的植物气息。岛上的树木参差不齐，地面上长满了各种奇异的花草。\n你并不清楚自己是如何来到这个神秘的小岛的。唯一的线索是身边的一封古老信件，上面写着一位已故亲戚的名字，而信中提到了一个传说中的古老遗迹，藏匿在岛的深处。信件的最后，有一段文字：“只有真正的冒险者才能解开这个谜团，而你，将成为我们的希望”。'},
 {'role': 'User', 'content': '进入丛林寻找可能的线索'},
 {'role': 'Bot',
  'content': '你决定进入丛林寻找线索。在丛林中，你发现了一些奇怪的痕迹，似乎是一种未知的生物留下的。你跟踪着这些痕迹，来到了一处空地。空地中央有一块石碑，上面刻着一些奇怪的符号。你仔细观察了一下，发现这些符号似乎是一种古老的文字，但你并不认识。'},
 {'role': 'User', 'content': '寻找岛上的居民询问石碑的来历'},
 {'role': 'Bot',
  'content': '你找到了一位年轻的男子，他告诉你这块石碑是为了记录岛上的历史和传说而刻石的。他告诉你，岛上有一个古老的文明，拥有强大的魔法和科技，但在一场古老的灾难中被分隔成数个孤立的岛屿。他还告诉你，古老文明的传说中，有一种可以打破能量屏障的秘密，而这个秘密被寄托在古老遗迹中。\n\n'}]